In [1]:
import sys
sys.path.append('/home/jgaud/data/PhD/Novelty Detection in Data Streams/streamndr-git/streamndr-private')
from streamndr.model import Minas, ECSMinerWF

import math
import numpy as np
from aim import Run, Distribution

from streamndr.metrics import ConfusionMatrixNovelty, MNew, FNew, ErrRate, CER, TTD, TTC
from river.metrics import MacroF1, Rand, AdjustedRand, MutualInfo, AdjustedMutualInfo, VBeta, FowlkesMallows
from sklearn.preprocessing import OneHotEncoder
from river import preprocessing
from river.utils import numpy2dict
import time
from scipy.io import arff
import pandas as pd
import numpy as np

from collections import deque
import math

In [2]:
def load_MOA(path):
    data = arff.loadarff(path)
    df = pd.DataFrame(data[0])
    
    for i, cl in enumerate(np.unique(df.iloc[:,-1])):
        df.iloc[:,-1].replace(cl, i, inplace=True)
    
    return df

## PAMAP2

In [3]:
# Load data
list_of_files = ['/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/pamap2+physical+activity+monitoring/PAMAP2_Dataset/Protocol/subject101.dat',
                 '/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/pamap2+physical+activity+monitoring/PAMAP2_Dataset/Protocol/subject102.dat',
                 '/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/pamap2+physical+activity+monitoring/PAMAP2_Dataset/Protocol/subject103.dat',
                 '/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/pamap2+physical+activity+monitoring/PAMAP2_Dataset/Protocol/subject104.dat',
                 '/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/pamap2+physical+activity+monitoring/PAMAP2_Dataset/Protocol/subject105.dat',
                 '/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/pamap2+physical+activity+monitoring/PAMAP2_Dataset/Protocol/subject106.dat',
                 '/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/pamap2+physical+activity+monitoring/PAMAP2_Dataset/Protocol/subject107.dat',
                 '/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/pamap2+physical+activity+monitoring/PAMAP2_Dataset/Protocol/subject108.dat',
                 '/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/pamap2+physical+activity+monitoring/PAMAP2_Dataset/Protocol/subject109.dat' ]

subjectID = [1,2,3,4,5,6,7,8,9]

activityIDdict = {0: 'transient',
              1: 'lying',
              2: 'sitting',
              3: 'standing',
              4: 'walking',
              5: 'running',
              6: 'cycling',
              7: 'Nordic_walking',
              9: 'watching_TV',
              10: 'computer_work',
              11: 'car driving',
              12: 'ascending_stairs',
              13: 'descending_stairs',
              16: 'vacuum_cleaning',
              17: 'ironing',
              18: 'folding_laundry',
              19: 'house_cleaning',
              20: 'playing_soccer',
              24: 'rope_jumping' }

colNames = ["timestamp", "activityID","heartrate"]

IMUhand = ['handTemperature', 
           'handAcc16_1', 'handAcc16_2', 'handAcc16_3', 
           'handAcc6_1', 'handAcc6_2', 'handAcc6_3', 
           'handGyro1', 'handGyro2', 'handGyro3', 
           'handMagne1', 'handMagne2', 'handMagne3',
           'handOrientation1', 'handOrientation2', 'handOrientation3', 'handOrientation4']

IMUchest = ['chestTemperature', 
           'chestAcc16_1', 'chestAcc16_2', 'chestAcc16_3', 
           'chestAcc6_1', 'chestAcc6_2', 'chestAcc6_3', 
           'chestGyro1', 'chestGyro2', 'chestGyro3', 
           'chestMagne1', 'chestMagne2', 'chestMagne3',
           'chestOrientation1', 'chestOrientation2', 'chestOrientation3', 'chestOrientation4']

IMUankle = ['ankleTemperature', 
           'ankleAcc16_1', 'ankleAcc16_2', 'ankleAcc16_3', 
           'ankleAcc6_1', 'ankleAcc6_2', 'ankleAcc6_3', 
           'ankleGyro1', 'ankleGyro2', 'ankleGyro3', 
           'ankleMagne1', 'ankleMagne2', 'ankleMagne3',
           'ankleOrientation1', 'ankleOrientation2', 'ankleOrientation3', 'ankleOrientation4']

columns = colNames + IMUhand + IMUchest + IMUankle  #all columns in one list

len(columns)

54

In [4]:
dataCollection = pd.DataFrame()
for file in list_of_files:
    procData = pd.read_table(file, header=None, sep='\s+')
    procData.columns = columns
    procData['subject_id'] = int(file[-5])
    dataCollection = dataCollection.append(procData, ignore_index=True)
    
dataCollection.reset_index(drop=True, inplace=True)
dataCollection.head()

/tmp/ipykernel_30344/2488565067.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataCollection = dataCollection.append(procData, ignore_index=True)
/tmp/ipykernel_30344/2488565067.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataCollection = dataCollection.append(procData, ignore_index=True)
/tmp/ipykernel_30344/2488565067.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataCollection = dataCollection.append(procData, ignore_index=True)
/tmp/ipykernel_30344/2488565067.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataCollection = dataCollection.append(procData, ignore_index=True)
/tmp/ipykernel_30344/2488565067.py:6

,timestamp,activityID,heartrate,handTemperature,handAcc16_1,handAcc16_2,handAcc16_3,handAcc6_1,handAcc6_2,handAcc6_3,...,ankleGyro2,ankleGyro3,ankleMagne1,ankleMagne2,ankleMagne3,ankleOrientation1,ankleOrientation2,ankleOrientation3,ankleOrientation4,subject_id
0,8.38,0,104.0,30.0,2.37223,8.60074,3.51048,2.43954,8.76165,3.35465,...,0.009250,-0.017580,-61.1888,-38.9599,-58.1438,1.0,0.0,0.0,0.0,1
1,8.39,0,NaN,30.0,2.18837,8.56560,3.66179,2.39494,8.55081,3.64207,...,-0.004638,0.000368,-59.8479,-38.8919,-58.5253,1.0,0.0,0.0,0.0,1
2,8.40,0,NaN,30.0,2.37357,8.60107,3.54898,2.30514,8.53644,3.73280,...,0.000148,0.022495,-60.7361,-39.4138,-58.3999,1.0,0.0,0.0,0.0,1
3,8.41,0,NaN,30.0,2.07473,8.52853,3.66021,2.33528,8.53622,3.73277,...,-0.020301,0.011275,-60.4091,-38.7635,-58.3956,1.0,0.0,0.0,0.0,1
4,8.42,0,NaN,30.0,2.22936,8.83122,3.70000,2.23055,8.59741,3.76295,...,-0.014303,-0.002823,-61.5199,-39.3879,-58.2694,1.0,0.0,0.0,0.0,1


In [5]:
def dataCleaning(dataCollection):
        dataCollection = dataCollection.drop(['handOrientation1', 'handOrientation2', 'handOrientation3', 'handOrientation4',
                                             'chestOrientation1', 'chestOrientation2', 'chestOrientation3', 'chestOrientation4',
                                             'ankleOrientation1', 'ankleOrientation2', 'ankleOrientation3', 'ankleOrientation4'],
                                             axis = 1)  # removal of orientation columns as they are not needed
        dataCollection = dataCollection.drop(dataCollection[dataCollection.activityID == 0].index) #removal of any row of activity 0 as it is transient activity which it is not used
        dataCollection = dataCollection.apply(pd.to_numeric, errors = 'coerce') #removal of non numeric data in cells
        dataCollection = dataCollection.interpolate() #removal of any remaining NaN value cells by constructing new data points in known set of data points
        
        return dataCollection

In [6]:
dataCol = dataCleaning(dataCollection)

In [7]:
dataCol.reset_index(drop = True, inplace = True)
dataCol.head(10)

,timestamp,activityID,heartrate,handTemperature,handAcc16_1,handAcc16_2,handAcc16_3,handAcc6_1,handAcc6_2,handAcc6_3,...,ankleAcc6_1,ankleAcc6_2,ankleAcc6_3,ankleGyro1,ankleGyro2,ankleGyro3,ankleMagne1,ankleMagne2,ankleMagne3,subject_id
0,37.66,1,NaN,30.375,2.21530,8.27915,5.58753,2.24689,8.55387,5.77143,...,9.63162,-1.76757,0.265761,0.002908,-0.027714,0.001752,-61.1081,-36.8636,-58.3696,1
1,37.67,1,NaN,30.375,2.29196,7.67288,5.74467,2.27373,8.14592,5.78739,...,9.58649,-1.75247,0.250816,0.020882,0.000945,0.006007,-60.8916,-36.3197,-58.3656,1
2,37.68,1,NaN,30.375,2.29090,7.14240,5.82342,2.26966,7.66268,5.78846,...,9.60196,-1.73721,0.356632,-0.035392,-0.052422,-0.004882,-60.3407,-35.7842,-58.6119,1
3,37.69,1,NaN,30.375,2.21800,7.14365,5.89930,2.22177,7.25535,5.88000,...,9.58674,-1.78264,0.311453,-0.032514,-0.018844,0.026950,-60.7646,-37.1028,-57.8799,1
4,37.70,1,100.0,30.375,2.30106,7.25857,6.09259,2.20720,7.24042,5.95555,...,9.64677,-1.75240,0.295902,0.001351,-0.048878,-0.006328,-60.2040,-37.1225,-57.8847,1
5,37.71,1,100.0,30.375,2.07165,7.25965,6.01218,2.19238,7.21038,6.01604,...,9.60177,-1.75239,0.311276,0.003793,-0.026906,0.004125,-61.3257,-36.9744,-57.7501,1
6,37.72,1,100.0,30.375,2.41148,7.59780,5.93915,2.23988,7.46679,6.03053,...,9.67694,-1.76748,0.326060,0.036814,-0.032277,-0.006866,-61.5520,-36.9632,-57.9957,1
7,37.73,1,100.0,30.375,2.32815,7.63431,5.70686,2.31663,7.64745,6.01495,...,9.61685,-1.76749,0.326380,-0.010352,-0.016621,0.006548,-61.5738,-36.1724,-59.3487,1
8,37.74,1,100.0,30.375,2.25096,7.78598,5.62821,2.28637,7.70801,5.93935,...,9.61686,-1.72212,0.326234,0.039346,0.020393,-0.011880,-61.7741,-37.1744,-58.1199,1
9,37.75,1,100.0,30.375,2.14107,7.52262,5.78141,2.31538,7.72276,5.78828,...,9.63189,-1.70699,0.326105,0.029874,-0.010763,0.005133,-60.7680,-37.4206,-58.8735,1


In [8]:
for i in range(0,4):
    dataCol["heartrate"].iloc[i]=100

/tmp/ipykernel_30344/1504362193.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataCol["heartrate"].iloc[i]=100


In [9]:
dataCol.isnull().sum()

timestamp           0
activityID          0
heartrate           0
handTemperature     0
handAcc16_1         0
handAcc16_2         0
handAcc16_3         0
handAcc6_1          0
handAcc6_2          0
handAcc6_3          0
handGyro1           0
handGyro2           0
handGyro3           0
handMagne1          0
handMagne2          0
handMagne3          0
chestTemperature    0
chestAcc16_1        0
chestAcc16_2        0
chestAcc16_3        0
chestAcc6_1         0
chestAcc6_2         0
chestAcc6_3         0
chestGyro1          0
chestGyro2          0
chestGyro3          0
chestMagne1         0
chestMagne2         0
chestMagne3         0
ankleTemperature    0
ankleAcc16_1        0
ankleAcc16_2        0
ankleAcc16_3        0
ankleAcc6_1         0
ankleAcc6_2         0
ankleAcc6_3         0
ankleGyro1          0
ankleGyro2          0
ankleGyro3          0
ankleMagne1         0
ankleMagne2         0
ankleMagne3         0
subject_id          0
dtype: int64

In [10]:
dataCol.sort_values(by=['timestamp'], inplace=True)

In [11]:
dataCol.head(10)

,timestamp,activityID,heartrate,handTemperature,handAcc16_1,handAcc16_2,handAcc16_3,handAcc6_1,handAcc6_2,handAcc6_3,...,ankleAcc6_1,ankleAcc6_2,ankleAcc6_3,ankleGyro1,ankleGyro2,ankleGyro3,ankleMagne1,ankleMagne2,ankleMagne3,subject_id
1936481,31.20,24,150.272727,24.875,-9.02648,3.43661,1.54825,-9.01793,3.42057,1.15826,...,6.27948,3.217450,-4.212500,-0.101032,-0.057742,-0.248516,-41.8305,-16.0762,5.25108,9
1936482,31.21,24,144.636364,24.875,-9.03504,3.35872,1.31746,-9.07594,3.51178,1.33933,...,5.79924,1.009420,-3.945540,-0.205677,-0.029978,0.074442,-41.4590,-15.8389,4.68478,9
1936483,31.22,24,139.000000,24.875,-9.05919,3.58967,1.66270,-9.12040,3.49709,1.44511,...,5.51824,-0.970548,-2.742000,-0.280073,0.034477,0.431696,-42.1015,-15.5489,4.82402,9
1936484,31.23,24,139.000000,24.875,-9.09083,3.70512,1.81579,-9.13307,3.63325,1.59583,...,5.02562,-1.816670,-1.934490,-0.240229,0.088297,0.643419,-41.7191,-15.8272,4.25836,9
1936485,31.24,24,139.000000,24.875,-9.12301,3.93405,1.93000,-9.19222,3.72433,1.64100,...,4.64947,-1.937920,-2.068320,0.123370,0.043536,0.780235,-42.2335,-15.4140,4.68156,9
1936486,31.25,24,139.000000,24.875,-9.04663,3.85792,1.96989,-9.15942,3.95071,1.73107,...,5.89815,-1.150670,-1.683970,0.457925,0.046488,0.948144,-42.6343,-14.4924,4.82092,9
1936487,31.26,24,139.000000,24.875,-8.65697,4.00945,2.28311,-9.02088,4.08583,1.94205,...,6.80227,-0.332962,-1.070860,0.633790,0.172421,1.141210,-42.6549,-13.5906,4.67797,9
1936488,31.27,24,139.000000,24.875,-8.46935,3.62893,2.32569,-8.74949,3.97827,2.06281,...,6.96891,0.151444,-0.770600,0.574341,0.264729,1.352240,-42.4006,-13.4741,4.82056,9
1936489,31.28,24,139.000000,24.875,-8.28094,3.55159,2.32878,-8.49355,3.84060,2.16855,...,7.04362,-0.136006,-0.845748,0.567359,0.300593,1.481750,-42.6647,-13.2043,4.53565,9
1936490,31.29,24,139.000000,24.875,-7.93601,3.47443,2.48815,-8.23698,3.73317,2.31954,...,7.44699,-0.938135,-1.329640,0.683415,0.374279,1.608790,-43.1795,-12.9207,4.67534,9


In [12]:
dataCol.set_index('timestamp', inplace=True)

In [13]:
dataCol

,activityID,heartrate,handTemperature,handAcc16_1,handAcc16_2,handAcc16_3,handAcc6_1,handAcc6_2,handAcc6_3,handGyro1,...,ankleAcc6_1,ankleAcc6_2,ankleAcc6_3,ankleGyro1,ankleGyro2,ankleGyro3,ankleMagne1,ankleMagne2,ankleMagne3,subject_id
timestamp,,,,,,,,,,,,,,,,,,,,,
31.20,24,150.272727,24.8750,-9.02648,3.43661,1.548250,-9.01793,3.42057,1.158260,0.555154,...,6.27948,3.217450,-4.21250,-0.101032,-0.057742,-0.248516,-41.8305,-16.07620,5.25108,9
31.21,24,144.636364,24.8750,-9.03504,3.35872,1.317460,-9.07594,3.51178,1.339330,0.446266,...,5.79924,1.009420,-3.94554,-0.205677,-0.029978,0.074442,-41.4590,-15.83890,4.68478,9
31.22,24,139.000000,24.8750,-9.05919,3.58967,1.662700,-9.12040,3.49709,1.445110,0.381105,...,5.51824,-0.970548,-2.74200,-0.280073,0.034477,0.431696,-42.1015,-15.54890,4.82402,9
31.23,24,139.000000,24.8750,-9.09083,3.70512,1.815790,-9.13307,3.63325,1.595830,0.253884,...,5.02562,-1.816670,-1.93449,-0.240229,0.088297,0.643419,-41.7191,-15.82720,4.25836,9
31.24,24,139.000000,24.8750,-9.12301,3.93405,1.930000,-9.19222,3.72433,1.641000,0.105799,...,4.64947,-1.937920,-2.06832,0.123370,0.043536,0.780235,-42.2335,-15.41400,4.68156,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4245.64,24,177.000000,28.5625,6.89559,6.13782,-0.608584,6.65593,6.59835,-0.329509,1.958890,...,9.53326,-1.497790,-1.73243,-0.135496,-0.045599,0.065027,-36.0814,5.58701,36.07410,2
4245.65,24,177.000000,28.5625,6.80903,6.47650,-0.995897,6.77081,6.29512,-0.706482,1.959530,...,9.51845,-1.422110,-1.68719,-0.115623,-0.038047,0.076966,-36.1900,5.47726,36.32140,2
4245.66,24,168.400000,28.5625,6.51286,7.16208,-0.964159,6.63710,6.70368,-0.888449,1.953730,...,9.53353,-1.406960,-1.67219,-0.090784,-0.005824,0.059216,-35.9658,5.46938,36.31950,2


In [14]:
dataCol.to_csv('/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/prepared_data/PAMAP2.csv', index=False)

## Room Occupancy

In [30]:
df = pd.read_csv('/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/Occupancy_Estimation.csv')

In [31]:
df

,Date,Time,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,2017/12/22,10:49:41,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,2017/12/22,10:50:12,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,2017/12/22,10:50:42,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,2017/12/22,10:51:13,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,2017/12/22,10:51:44,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10124,2018/01/11,08:58:07,25.06,25.13,24.69,25.31,6,7,33,22,0.09,0.04,0.06,0.08,345,0.000000,0,0,0
10125,2018/01/11,08:58:37,25.06,25.06,24.69,25.25,6,7,34,22,0.07,0.05,0.05,0.08,345,0.000000,0,0,0
10126,2018/01/11,08:59:08,25.13,25.06,24.69,25.25,6,7,34,22,0.11,0.05,0.06,0.08,345,0.000000,0,0,0
10127,2018/01/11,08:59:39,25.13,25.06,24.69,25.25,6,7,34,22,0.08,0.08,0.10,0.08,345,0.000000,0,0,0


In [32]:
object_columns = df.select_dtypes(include=['object']).columns.tolist()
print(object_columns)

['Date', 'Time']


In [33]:
df.drop(['Date', 'Time'], axis=1, inplace=True)

In [34]:
df

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10124,25.06,25.13,24.69,25.31,6,7,33,22,0.09,0.04,0.06,0.08,345,0.000000,0,0,0
10125,25.06,25.06,24.69,25.25,6,7,34,22,0.07,0.05,0.05,0.08,345,0.000000,0,0,0
10126,25.13,25.06,24.69,25.25,6,7,34,22,0.11,0.05,0.06,0.08,345,0.000000,0,0,0
10127,25.13,25.06,24.69,25.25,6,7,34,22,0.08,0.08,0.10,0.08,345,0.000000,0,0,0


In [35]:
object_columns = df.select_dtypes(include=['object']).columns.tolist()
print(object_columns)

[]


In [36]:
df.columns

Index(['S1_Temp', 'S2_Temp', 'S3_Temp', 'S4_Temp', 'S1_Light', 'S2_Light',
       'S3_Light', 'S4_Light', 'S1_Sound', 'S2_Sound', 'S3_Sound', 'S4_Sound',
       'S5_CO2', 'S5_CO2_Slope', 'S6_PIR', 'S7_PIR', 'Room_Occupancy_Count'],
      dtype='object')

In [37]:
df.Room_Occupancy_Count.value_counts()

0    8228
2     748
3     694
1     459
Name: Room_Occupancy_Count, dtype: int64

In [151]:
df.to_csv('/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/prepared_data/room_occupancy.csv', index=False)

## Credit Card

In [24]:
df = load_MOA('/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/creditcard.arff')

In [25]:
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [26]:
object_columns = df.select_dtypes(include=['object']).columns.tolist()
print(object_columns)

[]


In [27]:
df.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [29]:
df.Class.value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [156]:
df.to_csv('/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/prepared_data/creditcard.csv', index=False)

## Shuttle

In [20]:
colnames=['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'Class'] 
df_train = pd.read_csv('/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/shuttle.trn', names=colnames, header=None, delimiter=" ", index_col=0)
df_test = pd.read_csv('/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/shuttle.tst', names=colnames, header=None, delimiter=" ", index_col=0)

In [21]:
new_df = pd.concat([df_train, df_test]).sort_index(axis=0, ascending=True, inplace=False, kind='quicksort')

In [22]:
new_df

,X1,X2,X3,X4,X5,X6,X7,X8,Class
27,-105,78,0,42,10,51,37,-14,1
27,-63,78,0,42,8,51,37,-14,1
27,-227,78,0,42,13,51,37,-14,1
36,-5,75,0,36,0,39,39,0,1
36,-4,82,0,36,0,46,46,0,1
...,...,...,...,...,...,...,...,...,...
123,109,105,0,36,-1,-18,69,86,5
123,1409,105,1,36,-5,-18,69,86,5
123,213,105,0,36,-2,-18,69,86,5
123,136,105,0,36,-1,-18,69,86,5


In [23]:
new_df.Class.value_counts()

1    45586
4     8903
5     3267
3      171
2       50
7       13
6       10
Name: Class, dtype: int64

In [164]:
new_df.to_csv('/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/prepared_data/shuttle.csv', index=False)

## INSECTS

In [17]:
df = load_MOA('/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/INSECTS-abrupt_balanced_norm.arff')

In [84]:
df

,Att1,Att2,Att3,Att4,Att5,Att6,Att7,Att8,Att9,Att10,...,Att25,Att26,Att27,Att28,Att29,Att30,Att31,Att32,Att33,class
0,0.507066,0.153333,0.226092,0.302447,0.007239,0.369120,0.332436,0.017807,0.032819,0.033009,...,0.006855,0.017358,0.013430,0.009138,0.006768,0.007291,0.009224,0.036218,0.162955,2
1,0.281661,0.355953,0.253196,0.340335,0.415631,0.503923,0.392029,0.003648,0.068381,0.011155,...,0.005631,0.014048,0.002431,0.007076,0.037682,0.003089,0.004207,0.004144,0.005044,2
2,0.193750,0.257782,0.183339,0.247017,0.302133,0.363522,0.269729,0.293543,0.293002,0.029522,...,0.023837,0.013922,0.081406,0.413674,0.295615,0.120392,0.036566,0.032652,0.025776,4
3,0.514782,0.154867,0.016903,0.226084,0.297642,0.239111,0.248268,0.066745,0.115020,0.083407,...,0.020949,0.023019,0.021147,0.020813,0.019048,0.011606,0.013379,0.044839,0.123552,2
4,0.774337,0.012549,0.105751,0.033302,0.017170,0.049754,0.173500,0.055220,0.044184,0.034923,...,0.034876,0.060708,0.048119,0.027417,0.015022,0.010218,0.008121,0.012539,0.018058,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52843,0.376205,0.481608,0.341587,0.005883,0.555439,0.449868,0.535081,0.007482,0.004769,0.018222,...,0.027851,0.094857,0.095290,0.044267,0.008017,0.005021,0.006875,0.005422,0.008626,3
52844,0.907227,0.283415,0.401981,0.538330,0.006673,0.665257,0.644756,0.018815,0.014563,0.007784,...,0.004589,0.004021,0.004964,0.012643,0.050035,0.203125,0.130514,0.113380,0.038648,3
52845,0.417234,0.528923,0.374940,0.499970,0.001630,0.612408,0.594603,0.008641,0.006584,0.003708,...,0.004282,0.004831,0.026317,0.194038,0.200206,0.128182,0.032787,0.012716,0.010405,3
52846,0.379485,0.482561,0.341230,0.001819,0.442228,0.022463,0.446172,0.018756,0.012964,0.047095,...,0.073563,0.182334,0.221658,0.184219,0.053162,0.018325,0.011213,0.010824,0.020919,3


In [19]:
df['class'].value_counts()

2    8808
4    8808
1    8808
5    8808
0    8808
3    8808
Name: class, dtype: int64

In [85]:
object_columns = df.select_dtypes(include=['object']).columns.tolist()
print(object_columns)

[]


## NOAA

In [81]:
df = load_MOA('/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/NOAA.arff')

In [82]:
df

,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,class
0,19.8,14.0,1019.6,8.4,9.9,15.9,28.9,14.0,0
1,26.8,22.2,1006.2,8.1,10.9,19.0,34.0,21.0,0
2,34.6,32.9,1004.6,3.9,13.8,22.0,36.0,33.1,1
3,26.4,21.5,1006.9,8.0,18.7,30.1,39.9,16.0,1
4,14.7,7.9,1009.9,8.1,14.1,22.0,21.0,9.0,0
...,...,...,...,...,...,...,...,...,...
18154,32.6,21.9,1022.7,7.0,12.4,26.0,41.0,24.8,1
18155,36.8,25.4,1014.9,7.0,10.0,19.0,57.2,21.2,0
18156,41.9,29.9,1010.0,7.0,4.7,9.9,62.6,28.4,0
18157,42.4,29.7,1011.3,7.0,3.3,8.9,51.8,32.0,0


## Keystroke

In [18]:
df = load_MOA('/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/keystroke.arff')

In [19]:
df

,Att1,Att2,Att3,Att4,Att5,Att6,Att7,Att8,Att9,Att10,class
0,0.0978,0.0634,0.0430,0.6265,0.5054,0.1502,0.0798,0.0507,0.1945,0.1461,2
1,0.3723,0.5354,0.7529,0.2238,0.8424,0.4204,0.5123,0.1349,0.1892,0.5100,1
2,0.4129,0.3803,0.2497,0.4172,1.5137,0.2914,0.5746,0.2354,0.2969,0.8129,0
3,0.3417,0.6147,0.1807,0.6717,0.4537,0.2645,0.3585,0.1564,0.1203,0.2024,3
4,0.5834,0.5990,0.2483,0.4130,0.4506,0.2078,0.1035,0.1262,0.1607,0.4799,1
...,...,...,...,...,...,...,...,...,...,...,...
1595,0.0432,0.0185,-0.0306,0.0527,0.3356,-0.0246,0.0038,-0.0045,0.0799,0.0848,3
1596,0.1622,0.1731,0.0196,0.2103,0.2999,0.1768,-0.0092,0.0610,0.1615,0.3551,0
1597,0.0213,0.0673,0.0518,0.1417,0.2592,0.4558,0.0721,0.0283,0.0098,0.1648,2
1598,0.0150,0.0797,0.0628,0.1209,0.1935,0.1056,0.0605,0.0600,0.1565,0.3395,1


In [20]:
object_columns = df.select_dtypes(include=['object']).columns.tolist()
print(object_columns)

[]


In [21]:
df['class'].value_counts()

2    400
1    400
0    400
3    400
Name: class, dtype: int64

## Outdoor Objects

In [15]:
df = load_MOA('/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/outdoor.arff')

In [16]:
df

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att13,att14,att15,att16,att17,att18,att19,att20,att21,class
0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000936,0.013100,0.000312,...,0.780720,0.15783,0.000624,0.002807,0.002807,0.005303,0.000312,0.002807,0.002183,23
1,0.000289,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.001444,0.011265,0.000289,...,0.785960,0.15945,0.001155,0.000867,0.004333,0.007221,0.000000,0.002311,0.002600,23
2,0.000273,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.002460,0.019131,0.000000,...,0.797210,0.13966,0.000000,0.002460,0.001093,0.003006,0.000547,0.004099,0.003553,23
3,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000732,0.009277,0.000000,...,0.788330,0.15552,0.000488,0.000488,0.002441,0.006836,0.000000,0.003906,0.002441,23
4,0.000690,0.000000,0.0,0.000000,0.000000,0.000000,0.00023,0.001611,0.021629,0.001381,...,0.791070,0.13369,0.000460,0.003221,0.002071,0.005292,0.000230,0.002991,0.001381,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0.000000,0.000791,0.0,0.000198,0.000198,0.015628,0.00000,0.000000,0.016419,0.074975,...,0.000000,0.34639,0.473590,0.001780,0.000000,0.000989,0.000593,0.000000,0.000000,13
3996,0.000000,0.000000,0.0,0.000375,0.000938,0.015575,0.00000,0.000000,0.017639,0.075249,...,0.002815,0.34378,0.471380,0.002064,0.000000,0.002627,0.000563,0.000000,0.000000,13
3997,0.000000,0.000000,0.0,0.000152,0.002897,0.016164,0.00000,0.000000,0.013724,0.066484,...,0.006709,0.35590,0.471330,0.002592,0.000000,0.001220,0.001525,0.000000,0.000152,13
3998,0.000000,0.000000,0.0,0.000000,0.002198,0.015971,0.00000,0.000147,0.017289,0.063736,...,0.004542,0.34432,0.481610,0.002784,0.000000,0.001172,0.000440,0.000000,0.000000,13


In [17]:
df['class'].value_counts()

23    100
25    100
8     100
18    100
27    100
34    100
15    100
38    100
31    100
11    100
4     100
1     100
0     100
9     100
39    100
30    100
22    100
6     100
19    100
21    100
12    100
3     100
28    100
14    100
2     100
13    100
20    100
16    100
33    100
29    100
10    100
35    100
17    100
26    100
7     100
36    100
37    100
32    100
24    100
5     100
Name: class, dtype: int64

In [74]:
object_columns = df.select_dtypes(include=['object']).columns.tolist()
print(object_columns)

[]


## Rialto Bridge Timelapse

In [69]:
df = load_MOA('/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/rialto.arff')

In [70]:
df

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att19,att20,att21,att22,att23,att24,att25,att26,att27,class
0,0.98748,0.000037,0.0,0.000000,0.000018,0.0,0.0,0.0,0.0,0.000385,...,0.0,0.0,0.0,0.0,0.000073,0.000147,0.0,0.000000,0.005928,0
1,0.99887,0.000040,0.0,0.000000,0.000040,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,1
2,0.99364,0.000000,0.0,0.000023,0.000000,0.0,0.0,0.0,0.0,0.001081,...,0.0,0.0,0.0,0.0,0.000057,0.000000,0.0,0.000310,0.000621,2
3,0.99740,0.000104,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000291,...,0.0,0.0,0.0,0.0,0.000228,0.000000,0.0,0.000000,0.000000,3
4,1.00000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82245,0.99977,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000226,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,5
82246,0.99381,0.006186,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,6
82247,0.94239,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.018370,...,0.0,0.0,0.0,0.0,0.002120,0.000064,0.0,0.001734,0.005781,7
82248,0.91219,0.004011,0.0,0.000292,0.000146,0.0,0.0,0.0,0.0,0.029319,...,0.0,0.0,0.0,0.0,0.000802,0.000000,0.0,0.000365,0.000000,8


In [71]:
object_columns = df.select_dtypes(include=['object']).columns.tolist()
print(object_columns)

[]


## Power Supply

In [65]:
df = load_MOA('/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/powersupply.arff')

In [66]:
df

,attribute0,attribute1,class
0,117.4,127.0,0
1,139.1,126.0,1
2,128.0,120.0,12
3,127.0,112.0,17
4,122.8,110.0,18
...,...,...,...
29923,152.0,144.0,11
29924,148.0,140.0,13
29925,151.0,143.0,14
29926,168.0,161.0,15


In [67]:
object_columns = df.select_dtypes(include=['object']).columns.tolist()
print(object_columns)

[]


## Gas Sensor Array

In [9]:
df = load_MOA('/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/gassensor.arff')

In [10]:
df

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V120,V121,V122,V123,V124,V125,V126,V127,V128,Class
0,12285.6582,4.076635,4.842317,7.509393,10.822436,-1.312657,-1.853717,-6.924985,11800.9233,4.483500,...,-3.440387,1784.5324,1.907000,1.729200,4.881194,8.623828,-0.314110,-0.661556,-3.521663,3
1,-35.6889,0.993944,0.166099,0.489363,3.484663,-0.130298,-0.528364,-3.735347,266.4145,1.053988,...,-4.482534,904.9898,1.433707,1.068069,2.532958,5.369720,-0.183779,-0.534087,-4.635975,2
2,63927.2217,14.956941,19.971376,29.188512,33.291320,-10.433776,-16.062245,-49.490143,57405.8483,15.613843,...,-25.150090,14585.7879,8.189021,6.099452,12.127991,15.709651,-3.887082,-6.731473,-19.326895,3
3,2992.9019,1.380553,0.808910,1.288259,4.660135,-0.755903,-1.120470,-4.075213,4301.4033,1.652701,...,-7.215792,6044.5554,3.488295,2.662288,5.938297,8.544508,-1.567322,-2.701235,-6.472439,2
4,57524.7812,11.912566,14.631496,19.809240,23.715868,-9.084750,-11.770585,-39.234003,50051.0703,11.732548,...,-10.248794,10580.1006,5.752675,3.880740,8.545897,11.831716,-2.655521,-4.312744,-8.510591,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13905,13384.8262,2.820931,4.007378,6.618008,11.386095,-2.142994,-3.110327,-11.296786,19013.4575,4.050907,...,-8.012662,7548.3501,2.377836,3.400734,8.571174,11.555882,-1.617656,-2.457614,-6.226359,5
13906,13382.9619,2.825174,4.010915,6.483989,10.356127,-2.156512,-3.179563,-10.184803,19034.2495,4.066463,...,-6.943002,7510.4946,2.364505,3.401381,8.512949,12.149638,-1.613554,-2.493870,-6.859804,5
13907,13336.8725,2.822288,3.980818,6.487103,10.936979,-2.146688,-3.273109,-11.067489,18997.7222,4.055524,...,-7.777268,7530.0010,2.369898,3.400592,8.494436,11.839013,-1.612525,-2.504918,-6.263872,5
13908,13351.1318,2.824358,3.987819,6.554427,11.331002,-2.143651,-3.257854,-11.795109,19035.9926,4.071607,...,-6.890286,7599.0201,2.391834,3.358804,8.457260,11.297346,-1.606879,-2.438701,-6.044784,5


In [13]:
df['Class'].value_counts()

4    3009
1    2926
0    2565
3    1936
5    1833
2    1641
Name: Class, dtype: int64

In [60]:
object_columns = df.select_dtypes(include=['object']).columns.tolist()
print(object_columns)

[]


## Poker Hand

In [6]:
df = load_MOA('/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/poker-lsn.arff')

In [7]:
df

,s1,r1,s2,r2,s3,r3,s4,r4,s5,r5,class
0,b'1',1.0,b'1',10.0,b'1',11.0,b'1',12.0,b'2',13.0,4
1,b'1',1.0,b'1',10.0,b'1',11.0,b'1',12.0,b'4',13.0,4
2,b'1',1.0,b'1',10.0,b'1',11.0,b'2',11.0,b'1',12.0,1
3,b'1',1.0,b'1',10.0,b'1',11.0,b'2',11.0,b'1',13.0,1
4,b'1',1.0,b'1',10.0,b'1',11.0,b'2',11.0,b'2',12.0,1
...,...,...,...,...,...,...,...,...,...,...,...
829196,b'4',9.0,b'4',11.0,b'3',12.0,b'4',12.0,b'1',13.0,1
829197,b'4',9.0,b'4',11.0,b'3',12.0,b'4',12.0,b'3',13.0,1
829198,b'4',9.0,b'4',11.0,b'3',12.0,b'4',12.0,b'4',13.0,1
829199,b'4',9.0,b'4',11.0,b'4',12.0,b'1',13.0,b'2',13.0,1


In [8]:
df['class'].value_counts()

0    415526
1    350426
2     39432
3     17541
4      3225
5      1657
6      1186
7       195
8        11
9         2
Name: class, dtype: int64

In [167]:
object_columns = df.select_dtypes(include=['object']).columns.tolist()
print(object_columns)

['s1', 's2', 's3', 's4', 's5']


In [168]:
for col in object_columns:
    df[col] = df[col].str.decode('utf-8') 

In [169]:
df

,s1,r1,s2,r2,s3,r3,s4,r4,s5,r5,class
0,1,1.0,1,10.0,1,11.0,1,12.0,2,13.0,4
1,1,1.0,1,10.0,1,11.0,1,12.0,4,13.0,4
2,1,1.0,1,10.0,1,11.0,2,11.0,1,12.0,1
3,1,1.0,1,10.0,1,11.0,2,11.0,1,13.0,1
4,1,1.0,1,10.0,1,11.0,2,11.0,2,12.0,1
...,...,...,...,...,...,...,...,...,...,...,...
829196,4,9.0,4,11.0,3,12.0,4,12.0,1,13.0,1
829197,4,9.0,4,11.0,3,12.0,4,12.0,3,13.0,1
829198,4,9.0,4,11.0,3,12.0,4,12.0,4,13.0,1
829199,4,9.0,4,11.0,4,12.0,1,13.0,2,13.0,1


In [170]:
df.to_csv('/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/prepared_data/poker.csv', index=False)

## Sensor Stream

In [3]:
df = load_MOA('/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/sensorstream.arff')

In [4]:
df

,rcdminutes,temperature,humidity,light,voltage,class
0,58.0,19.7336,37.0933,71.76000,2.69964,10
1,58.0,19.6160,37.5737,97.52000,2.69964,17
2,58.0,19.7728,37.1620,143.52000,2.71196,16
3,58.0,20.2040,36.8871,50.60000,2.69964,21
4,58.0,20.4196,36.6118,121.44000,2.65143,51
...,...,...,...,...,...,...
2219798,660.0,23.4968,35.8530,0.04022,2.81479,49
2219799,661.0,23.4968,35.8530,0.04022,2.81479,49
2219800,661.0,23.4772,35.8530,0.04022,2.82819,49
2219801,662.0,23.4968,35.8530,0.04022,2.81479,49


In [5]:
df['class'].value_counts()

23    65689
20    64384
15    62409
18    61513
14    60164
13    58521
41    58215
16    57352
40    56858
28    56357
52    55354
38    53245
17    53162
39    52988
27    51338
30    49155
26    48764
29    47915
37    47681
1     47155
11    46915
21    46633
33    46621
54    45204
35    44860
32    43793
24    43092
0     43047
44    42259
2     41833
34    40836
10    39455
36    38656
22    38343
19    37627
25    35749
51    35666
42    34811
45    34067
8     33779
9     33433
7     32998
31    32736
12    28832
47    28718
4     27013
5     26667
46    25622
3     19016
53    15809
43    15737
50     4497
48     2850
49     2372
6      2038
Name: class, dtype: int64

## Ozone

In [12]:
df = load_MOA('/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/ozone.arff')

In [13]:
df

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V64,V65,V66,V67,V68,V69,V70,V71,V72,Class
0,0.8,1.8,2.4,2.1,2.0,2.1,1.5,1.7,1.9,2.3,...,0.150000,10.670000,-1.560000,5795.000000,-12.100000,17.900000,10330.000000,-55.000000,0.00,0
1,2.8,3.2,3.3,2.7,3.3,3.2,2.9,2.8,3.1,3.4,...,0.480000,8.390000,3.840000,5805.000000,14.050000,29.000000,10275.000000,-55.000000,0.00,0
2,2.9,2.8,2.6,2.1,2.2,2.5,2.5,2.7,2.2,2.5,...,0.600000,6.940000,9.800000,5790.000000,17.900000,41.300000,10235.000000,-40.000000,0.00,0
3,4.7,3.8,3.7,3.8,2.9,3.1,2.8,2.5,2.4,3.1,...,0.490000,8.730000,10.540000,5775.000000,31.150000,51.700000,10195.000000,-40.000000,2.08,0
4,2.6,2.1,1.6,1.4,0.9,1.5,1.2,1.4,1.3,1.4,...,0.304716,9.872418,0.830116,5818.821222,10.511051,37.388335,10164.198442,-0.119949,0.58,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2529,0.3,0.4,0.5,0.5,0.2,0.3,0.4,0.4,1.3,2.2,...,0.070000,7.930000,-4.410000,5800.000000,-25.600000,21.800000,10295.000000,65.000000,0.00,0
2530,1.0,1.4,1.1,1.7,1.5,1.7,1.8,1.5,2.1,2.4,...,0.040000,5.950000,-1.140000,5845.000000,-19.400000,19.100000,10310.000000,15.000000,0.00,0
2531,0.8,0.8,1.2,0.9,0.4,0.6,0.8,1.1,1.5,1.5,...,0.060000,7.800000,-0.640000,5845.000000,-9.600000,35.200000,10275.000000,-35.000000,0.00,0
2532,1.3,0.9,1.5,1.2,1.6,1.8,1.1,1.0,1.9,2.0,...,0.250000,7.720000,-0.890000,5845.000000,-19.600000,34.200000,10245.000000,-30.000000,0.05,0


In [14]:
df['Class'].value_counts()

0    2374
1     160
Name: Class, dtype: int64

## Luxembourg

In [3]:
df = load_MOA('/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/luxembourg.arff')

In [4]:
object_columns = df.select_dtypes(include=['object']).columns.tolist()
print(object_columns)

['att6', 'att7', 'att18', 'att19', 'att20', 'att21', 'att22', 'att23', 'att24', 'att25', 'att26', 'att27', 'att28', 'att29', 'att30']


In [5]:
for col in object_columns:
    df[col] = df[col].str.decode('utf-8') 

In [6]:
df

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att23,att24,att25,att26,att27,att28,att29,att30,att31,class
0,0.57143,0.14286,1.00000,0.5,0.6,1,1,0.2,0.16667,0.50,...,0,0,0,0,0,0,0,0,0.20863,1
1,0.57143,0.14286,0.66667,0.6,0.5,1,1,1.0,1.00000,0.00,...,0,0,0,0,0,0,0,0,0.20923,1
2,0.71429,0.28571,0.00000,0.8,0.9,0,1,0.8,0.50000,0.00,...,0,0,0,0,0,0,0,0,0.20968,0
3,0.85714,0.28571,0.33333,0.3,0.4,1,1,0.7,0.16667,0.50,...,0,0,0,0,0,0,0,0,0.20973,0
4,0.42857,0.14286,0.66667,0.6,0.0,1,1,0.6,0.50000,0.25,...,0,0,0,0,0,0,0,0,0.21002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1896,0.28571,0.00000,0.00000,0.7,0.5,0,1,0.8,0.66667,0.25,...,0,0,0,0,0,0,0,0,0.50964,0
1897,0.42857,0.00000,0.66667,0.5,0.5,1,1,0.7,0.66667,0.00,...,0,0,0,0,0,0,0,0,0.51145,1
1898,0.57143,0.14286,0.66667,0.4,0.4,1,1,0.9,0.66667,0.25,...,0,0,0,0,0,0,0,0,0.51157,1
1899,0.42857,0.14286,0.33333,0.0,0.1,1,0,1.0,0.83333,0.00,...,1,1,1,1,1,1,1,1,0.51158,0


In [175]:
df.to_csv('/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/prepared_data/luxembourg.csv', index=False)

In [23]:
df['class'].value_counts()[0] < 500

True

## Electricity

In [9]:
df = load_MOA('/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/USP_repository/elec.arff')

In [10]:
df

,date,day,period,nswprice,nswdemand,vicprice,vicdemand,transfer,class
0,0.0000,b'2',0.000000,0.056443,0.439155,0.003467,0.422915,0.414912,1
1,0.0000,b'2',0.021277,0.051699,0.415055,0.003467,0.422915,0.414912,1
2,0.0000,b'2',0.042553,0.051489,0.385004,0.003467,0.422915,0.414912,1
3,0.0000,b'2',0.063830,0.045485,0.314639,0.003467,0.422915,0.414912,1
4,0.0000,b'2',0.085106,0.042482,0.251116,0.003467,0.422915,0.414912,0
...,...,...,...,...,...,...,...,...,...
45307,0.9158,b'7',0.914894,0.044224,0.340672,0.003033,0.255049,0.405263,0
45308,0.9158,b'7',0.936170,0.044884,0.355549,0.003072,0.241326,0.420614,0
45309,0.9158,b'7',0.957447,0.043593,0.340970,0.002983,0.247799,0.362281,0
45310,0.9158,b'7',0.978723,0.066651,0.329366,0.004630,0.345417,0.206579,1


In [11]:
df['class'].value_counts()

0    26075
1    19237
Name: class, dtype: int64

In [180]:
new_df = pd.get_dummies(df, columns=['day', ])

In [181]:
new_df

,date,period,nswprice,nswdemand,vicprice,vicdemand,transfer,class,day_b'1',day_b'2',day_b'3',day_b'4',day_b'5',day_b'6',day_b'7'
0,0.0000,0.000000,0.056443,0.439155,0.003467,0.422915,0.414912,1,0,1,0,0,0,0,0
1,0.0000,0.021277,0.051699,0.415055,0.003467,0.422915,0.414912,1,0,1,0,0,0,0,0
2,0.0000,0.042553,0.051489,0.385004,0.003467,0.422915,0.414912,1,0,1,0,0,0,0,0
3,0.0000,0.063830,0.045485,0.314639,0.003467,0.422915,0.414912,1,0,1,0,0,0,0,0
4,0.0000,0.085106,0.042482,0.251116,0.003467,0.422915,0.414912,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45307,0.9158,0.914894,0.044224,0.340672,0.003033,0.255049,0.405263,0,0,0,0,0,0,0,1
45308,0.9158,0.936170,0.044884,0.355549,0.003072,0.241326,0.420614,0,0,0,0,0,0,0,1
45309,0.9158,0.957447,0.043593,0.340970,0.002983,0.247799,0.362281,0,0,0,0,0,0,0,1
45310,0.9158,0.978723,0.066651,0.329366,0.004630,0.345417,0.206579,1,0,0,0,0,0,0,1


In [182]:
new_df.to_csv('/home/jgaud/data/PhD/Novelty Detection in Data Streams/Data/prepared_data/elec.csv', index=False)